The purpose of this notebook is to quantify bacterial cells in a set of environmental samples. We can start by importing the analysis packages we need. 

In [1]:
import os
import FlowCytometryTools as fct
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_dir = "../FlowCytometryData/FCS/2019-10-30 Keith Arora-Williams Bead Test"
data_files = {i[:-8]:os.path.join(data_dir, i) for i in os.listdir(data_dir)}

bead_sample = fct.FCMeasurement(ID='just_beads', datafile=data_files['Specimen_001_countbright beads'])
bead_sample_df = bead_sample.data
beads_and_cells = fct.FCMeasurement(ID='F_Mix_R1_SG1', datafile=data_files['Specimen_001_Tube 4'])
beads_and_cells_df = beads_and_cells.data
sample = fct.FCMeasurement(ID='F_Contam_R1_SG1', datafile=data_files['Specimen_001_Tube 1'])
tube1_df = sample.data
sample = fct.FCMeasurement(ID='F_Mix_R1_SG0', datafile=data_files['Specimen_001_Tube 2'])
tube2_df = sample.data
sample = fct.FCMeasurement(ID='F_Media_R1', datafile=data_files['Specimen_001_Tube 7'])
tube7_df = sample.data
sample = fct.FCMeasurement(ID='UF_Mix_R1', datafile=data_files['Threshold SSC 200_Tube 9'])
tube9_df = sample.data
sample = fct.FCMeasurement(ID='F_Mix_R1_SG1', datafile=data_files['Threshold SYBR Green 1000_Tube 4'])
tube4_df = sample.data
print(tube4_df.columns)

In [ ]:
import seaborn as sns

data_df  = bead_sample_df.copy()
if len(data_df) <= 10000:
    data_df_s = data_df.copy()
else:
    data_df_s = dat_df.sample(10000)

sns.set(style="whitegrid", color_codes=True)
print("Plotting scatter plots")
scatter_pairs = [('488 530/30-A', 'SSC-A'), ('488 530/30-A', '488 585/42-A'),
                 ('488 530/30-A', '488 670LP-A'), ('488 530/30-A', '488 780/60-A')]
figs, axes_s = plt.subplots(2,2, figsize=(12, 12), num=2)
figs.suptitle(name)
for (c1, c2), x, y in zip(scatter_pairs, [0,0,1,1], [0,1,0,1]):
    g = sns.jointplot(ch1, ch2, data=data_df_s, color="k",).plot_joint(sns.kdeplot, zorder=0, n_levels=10)

plt.show()

In [ ]:
channels = ['SSC-A', '488 530/30-A', '488 585/42-A', "488 670LP-A", "488 780/60-A"]
channels = [i for i in bead_sample_df.columns if i.endswith("A")]
colors_ = ['red', 'green', 'blue', 'magenta', 'cyan', 'black', 'yellow', 'orange']
plt.close('all');
fig, axes_ = plt.subplots(len(channels), 1, figsize=(12,2.5*len(channels)), num=1)
for idx, (ch, co) in enumerate(zip(channels, colors_)):
    print(ch, co, bead_sample_df[ch].shape, bead_sample_df[ch].min(), bead_sample_df[ch].max())
    bead_sample_df[ch].plot.hist(bins=int(bead_sample_df.shape[0]/5), color=co, alpha=0.7, ax=axes_[idx])
    axes_[idx].set_ylim(ymin=0, ymax=50)

In [ ]:
fig, axes_ = plt.subplots(len(channels), 1, figsize=(12,2.5*len(channels)), num=2)
for idx, (ch, co) in enumerate(zip(channels, colors_)):
    print(ch, co, beads_and_cells_df[ch].shape, beads_and_cells_df[ch].min(), bead_sample_df[ch].max())
    beads_and_cells_df[ch].plot.hist(bins=int(800), color=co, alpha=0.7, ax=axes_[idx])
    axes_[idx].set_ylim(ymin=0, ymax=50)
#    axes_[idx].set_xlim(xmin=-500, xmax=25000)

In [ ]:
#bead_threshold = {'SSC-A': [200000, 350000],


fixative_dilution = 0.5 # 
bead_concentration = 1e6 # beads per mL
bead_volume = 0.05 # mL
cells_volume = 0.5 # mL 
ratio_multiplier = (bead_concentration*bead_volume)/(cells_volume)
df_ls = [bead_sample_df, beads_and_cells_df, tube4_df, tube1_df, tube2_df, tube7_df, tube9_df]
name_ls = ['just_beads', 'beads_and_cells', "Tube4", "Tube1", "Tube2", "Tube7", "Tube9"]
for df, df_n in zip(df_ls, name_ls):
    bead_idx = None
    for ch, (l1, l2) in bead_threshold.items():
        if not bead_idx:
            bead_idx = set(df[(df[ch] > l1) & (df[ch] < l2)].index)
        else:
            bead_idx = bead_idx.intersection(set(df[(df[ch] > l1) & (df[ch] < l2)].index))
    cell_idx = None       
    for ch, (l1, l2) in cell_threshold.items():
        if not cell_idx:
            cell_idx = set(df[(df[ch] > l1) & (df[ch] < l2)].index)
        else:
            cell_idx = cell_idx.intersection(set(df[(df[ch] > l1) & (df[ch] < l2)].index))

    if df_n == 'Tube9':
        cell_idx = set(df.index) - cell_idx - bead_idx
    other_particles = df.shape[0] - len(bead_idx) - len(cell_idx)
    assert len(bead_idx.intersection(cell_idx)) == 0
    event_ratio = len(cell_idx)/len(bead_idx) # cells/bead
    concentration = (ratio_multiplier*event_ratio)/fixative_dilution # cells/bead * beads/mL = cells/mL
    print("Sample: {} has {} beads, {} cells, {} other events".format(df_n, len(bead_idx), len(cell_idx), other_particles))
    print("Cell concentration is {:.3e} cells/mL".format(concentration))
    